In [2]:
from elasticsearch import Elasticsearch
import csv

es = Elasticsearch([{'host':'nyuvis-web.poly.edu', 'port': 80, 'url_prefix':'es'}])

In [3]:
sample_res = es.search(index = "nips_papers", body = { "size": 25})
sample_res

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '1001',
    '_index': 'nips_papers',
    '_score': 1.0,
    '_source': {'abstract': 'Abstract Missing',
     'event_type': '',
     'id': '1001',
     'paper_text': 'Neural Network Ensembles, Cross\nValidation, and Active Learning\n\nAnders Krogh"\nNordita\nBlegdamsvej 17\n2100 Copenhagen, Denmark\n\nJesper Vedelsby\nElectronics Institute, Building 349\nTechnical University of Denmark\n2800 Lyngby, Denmark\n\nAbstract\nLearning of continuous valued functions using neural network ensembles (committees) can give improved accuracy, reliable estimation of the generalization error, and active learning. The ambiguity\nis defined as the variation of the output of ensemble members averaged over unlabeled data, so it quantifies the disagreement among\nthe networks. It is discussed how to use the ambiguity in combination with cross-validation to give a reliable estimate of the ensemble\ngeneralizatio

In [8]:
#multi-set analysis: each year in aggregation of years contains an aggregation of significant terms
sig_terms_res = es.search(index = "nips_papers", body = {
    "size": 0,
    "aggs": {
        "keywords_by_years": {
            "terms": {
                "field": "year",
                "size": 50
            },
            "aggs": {
                "keywords": {
                    "significant_terms": {
                        "field": "title",
                        "size": 50
                    }
                }
            }
        }
    }
})

lst = [] #flatten the dictionary into a list

sig_dict = sig_terms_res['aggregations']['keywords_by_years']['buckets']

for year in sig_dict:
    words = year['keywords']['buckets']
    for word in words:
        line = {
            'score': word['score'],
            'year': year['key'],
            'doc_count': word['doc_count'],
            'keyword': word['key']
        }
        lst.append(line)

with open('sig_terms_res.csv', 'w', newline='') as f:
    header_present = False
    for doc in lst:
        if not header_present:
            w = csv.DictWriter(f, doc.keys())
            w.writeheader()
            header_present = True
        w.writerow(doc)

In [9]:
#multi-set analysis: each year in aggregation of years contains an aggregation of general (total) terms
tot_terms_res = es.search(index = "nips_papers", body = {
    "size": 0,
    "aggs": {
        "keywords_by_years": {
            "terms": {
                "field": "year",
                "size": 50
            },
            "aggs": {
                "keywords": {
                    "terms": {
                        "field": "title",
                        "size": 50
                    }
                }
            }
        }
    }
})

lst2 = [] #flatten the dictionary into a list

tot_dict = tot_terms_res['aggregations']['keywords_by_years']['buckets']

for year in tot_dict:
    words = year['keywords']['buckets']
    for word in words:
        line = {
            'year': year['key'],
            'doc_count': word['doc_count'],
            'keyword': word['key']
        }
        lst2.append(line)

with open('tot_terms_res.csv', 'w', newline='') as f:
    header_present = False
    for doc in lst2:
        if not header_present:
            w = csv.DictWriter(f, doc.keys())
            w.writeheader()
            header_present = True
        w.writerow(doc)